In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import textract
import PyPDF2
import en_core_web_sm
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
import spacy
import os
import io
import requests
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output

print(f"PyTorch version: {torch.__version__}")

%matplotlib inline

PyTorch version: 1.12.1+cu102


In [233]:
DEBUG           = False

INPUT_DIR       = 'articles'

#USE_APEX        = True
#APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 768  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

# if USE_APEX:
#     TRAIN_BATCHSIZE = 4
#     BATCH_UPDATE    = 16
# else:
TRAIN_BATCHSIZE = 2
BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [234]:
# Load pre-trained English language model
# nlp = nl_core_news_sm.load()
nlp = spacy.load("en_core_web_sm")

# File Extension. set as 'pdf' or as 'doc(x)'
extension = 'pdf'
diro='/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles'

In [235]:
def extract_text_from_pdf(file):
    '''Opens and reads in a PDF file from path'''
    
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    page_count = fileReader.getNumPages()
    text = [fileReader.getPage(i).extractText() for i in range(page_count)]
    
    return str(text).replace("\\n", "")

def extract_text_from_word(filepath):
    '''Opens and reads in a .doc or .docx file from path'''
    
    txt = textract.process(filepath).decode('utf-8')
    
    return txt.replace('\n', ' ').replace('\t', ' ')


In [236]:
def create_tokenized_texts_list(extension):
    '''Create two lists, one with the names of the candidate and one with the tokenized 
       resume texts extracted from either a .pdf or .doc'''
    resume_texts, resume_names = [], []
    
    # Loop over the contents of the directory containing the resumes, filtering by .pdf or .doc(x)
    for resume in list(filter(lambda x: extension in x, os.listdir(diro + '/CV'))):
        if extension == 'pdf':
            # Read in every resume with pdf extension in the directory
            resume_texts.append(nlp(extract_text_from_pdf(diro + '/CV/' + resume)))
        elif 'doc' in extension:
            # Read in every resume with .doc or .docx extension in the directory
            resume_texts.append(nlp(extract_text_from_word(diro + '/CV/' + resume)))
            
        resume_names.append(resume.split('_')[0].capitalize())
        
    return resume_texts, resume_names

In [237]:
def add_newruler_to_pipeline(skill_pattern_path):
    '''Reads in all created patterns from a JSONL file and adds it to the pipeline after PARSER and before NER'''
    
    new_ruler = EntityRuler(nlp).from_disk(skill_pattern_path)
    nlp.add_pipe(new_ruler, after='parser')
    

def visualize_entity_ruler(entity_list, doc):
    '''Visualize the Skill entities of a doc'''
    
    options = {"ents": entity_list}
    displacy.render(doc, style='ent', options=options)
   

In [238]:
skillz = "/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles/Skill_List/skill_patterns.jsonl"
with jsonlines.open(skillz) as f:
    created_entities = [line['label'].upper() for line in f.iter()]

ruler = EntityRuler(nlp).from_disk(skillz)
nlp.add_pipe(ruler, after='parser')

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_=="SKILL":
             subset.append(ent.text)
    myset.append(subset)
    return subset

In [239]:
texer = open("/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles/Cover_Letters/UI_UX Designer - 1000 tokens.txt")
texo = open("/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles/Cover_Letters/Digital Marketing Consultant - 500 tokens.txt")
datto = texer.read()
datum = texo.read()

texo.close()
texer.close()

In [258]:


cha = extract_text_from_pdf("/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles/Resume/Resume.pdf")

#open text file in read mode
text_file = open("/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/Workfiles/Cover_Letters/Data Specialist-500 tokens.txt", "r")
 
#read whole file to a string
data = text_file.read()

#close file
text_file.close()

pb = pd.DataFrame(
    columns=['title','body']
    )
pb.loc[0]='Data Specialist',data
pb.loc[1]='Idiot', cha
pb.loc[2]='UX_UI Designer', datum
pb.loc[3]='Digital Marketing Consultant', datto

In [259]:
pb.head()

,title,body
0,Data Specialist,Beginning - manual input:\nDear Ms. Mustermann...
1,Idiot,"['Samuel SpearingFairfield, Connecticut • (203..."
2,UX_UI Designer,Beginning - manual input:\nDear Ms. Mustermann...
3,Digital Marketing Consultant,Beginning - manual input:\nDear Ms. Mustermann...


In [260]:
pb['skills']=pb['body'].apply(get_skills)

In [261]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [262]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer, randomize=True):

        title, text, keywords = [], [], []
        for v in range(len(data)):
            title.append(data['title'][v])
            text.append(data['body'][v])
            keywords.append(data['skills'][v])

        self.randomize = randomize
        self.tokenizer = tokenizer 
        self.title     = title
        self.text      = text
        self.keywords  = keywords  

    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

    def __len__(self):
        return len(self.text)
    
    def __getBit(self):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)
        return kw
    
    def __getitem__(self, i):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)
        
        input = SPECIAL_TOKENS['bos_token'] + self.title[i] + \
                SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token'] + \
                self.text[i] + SPECIAL_TOKENS['eos_token']

        encodings_dict = tokenizer(input,                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids), 
                'attention_mask': torch.tensor(attention_mask)}

In [263]:
ivso = list(pb.keys())
idso = ivso[:2+1]
ibso = ivso[2:]
print(idso)

['title', 'body', 'skills']


In [264]:
def split_data(data, S=TRAIN_SIZE):
    # Shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # Split into training and validation sets    
    train_size = int(S * len(data))

    train_ids = ids[:train_size+1]
    val_ids = ids[train_size:]

    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]

    return train_data, val_data

In [265]:
data = pb.to_dict()
ids = list(data.keys())
print(ids[:3+1])
# bobi, zobi = split_data(pb.to_dict())
# bobi.items()

['title', 'body', 'skills']


In [266]:
data = pb.to_dict()
k, d = split_data(data)
b = []
for i in range(3):
    print(k['title'][0])

Data Specialist
Data Specialist
Data Specialist


In [267]:
title, text, keywords = [], [], []
for v in range(len(data)):
            title.append(data['title'][v])
            text.append(data['body'][v])
            keywords.append(data['skills'][v])
print(len(keywords.copy()))

3


In [268]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    #model.cuda()
    return model

In [269]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/absconditus/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_

Special tokens added


loading weights file https://huggingface.co/gpt2/resolve/main/pytorch_model.bin from cache at /home/absconditus/.cache/huggingface/transformers/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


CPU times: user 6.81 s, sys: 479 ms, total: 7.29 s
Wall time: 3.7 s


In [270]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

In [272]:
train_data, val_data = split_data(pb.to_dict())

print(val_data)

{'body': {0: 'Beginning - manual input:\nDear Ms. Mustermann, with this letter and attached resume, I would like to express my sincere interest in the vacancy for ‘Data Specialist’."\n===============================================================================================================================================================\n\nAs a highly skilled and highly accomplished professional with comprehensive expertise in performing all facets of data entry and data management, I am confident that I would significantly contribute to the success of your organization.\n\nMy career accomplishments include more than four years of experience in positions of increasing responsibility within the Data Entry and Analysis Center at Juniper Systems. In addition to this, my comprehensive knowledge of quality assurance and storage techniques is certain to render me an asset to your organization. Furthermore, my superior communication and problem-solving skills position me to make a signif

In [273]:
train_data, val_data = split_data(pb.to_dict())

train_dataset = myDataset(train_data, tokenizer)
##future try using val data instead of train data here
val_dataset = myDataset(train_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

'There are 3 samples for training, and 3 samples for validation testing'

In [ ]:
%%time

training_args = TrainingArguments(
    output_dir="/home/absconditus/Documents/Github/Python Projects/jupyter/TF LSTM RNN for cover letter generation/resulto",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="steps",
    #fp16=True,
    #fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=True,     
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

#---------------------------------------------------#
trainer.train()
trainer.save_model()    

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/absconditus/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3
  Num Epochs = 4
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 32
  Total optimization steps = 4


Step,Training Loss,Validation Loss


In [ ]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS)
                  #load_model_path='pytorch_model.bin')

In [23]:
title = "We got a lot of grief when our photo became a meme"
keywords = ['train', 'lads', 'drinking', 'picture', 'funny', 'instagram']
kw = myDataset.join_keywords(keywords, randomize=False)

prompt = SPECIAL_TOKENS['bos_token'] + title + \
         SPECIAL_TOKENS['sep_token'] + kw + SPECIAL_TOKENS['sep_token']
         
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

RuntimeError: No CUDA GPUs are available

In [24]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=50, 
                                max_length=MAXLEN,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(','.join(keywords))    
    print("{}: {}\n\n".format(i+1,  text[a:]))

NameError: name 'model' is not defined

In [ ]:
# Beam-search text generation:
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(','.join(keywords))    
    print("{}: {}\n\n".format(i+1,  text[a:]))

In [ ]:
tokenizer = get_tokenier()
model = get_model(tokenizer)

In [ ]:
prompt = title

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval()
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))